# Movebank Data Analysis Attempt

## Part 1: Basic Manipulation

### Checking up the data and 1st Degree Clean-up: Identify the Useful Columns

After downloading the data, I found that the package came with two csv files. First, we can import the two tables - reference and Argos. To do so, we will have to import pandas (and numpy and matplotlib, for later use).

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

ImportError: dlopen(/Users/s.n.zhou/anaconda3/lib/python3.6/site-packages/matplotlib/ft2font.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libfreetype.6.dylib
  Referenced from: /Users/s.n.zhou/anaconda3/lib/python3.6/site-packages/matplotlib/ft2font.cpython-36m-darwin.so
  Reason: Incompatible library version: ft2font.cpython-36m-darwin.so requires version 22.0.0 or later, but libfreetype.6.dylib provides version 21.0.0

Read the reference table

In [ ]:
whale_ref = pd.read_csv("../data/Blue_whales_reference.csv")
whale_ref.head()

In [ ]:
whale_ref.shape

The reference table records the information of each blue whale monitore. It is of size 143 rows x 16 columns. Let's figure out what each column is about.

In [ ]:
whale_ref.columns

Combined with the column information provided in the README.txt file, we can find a few columns that provide informations that will be useful to us 
- *amimal-id*: An individual identifier for the subjects monitored
- *deploy-on-date* & *deploy-off-date*: the start and end time of monitoring
- *deploy-on-latitude* & *deploy-on-longitutde*: the start location of monitoring
- *study-site*: name of deployment site/facility

Before we downsize the table to a more concise version, let's check if the some standard measures (i.e. other less interesting columns) are uniform for each subject recorded.

In [ ]:
whale_ref.groupby(['animal-life-stage','animal-taxon','attachment-type',
                   'manipulation-type','tag-readout-method']).count()

In [ ]:
whale_ref.groupby(['tag-manufacturer-name','tag-model']).count()

Although most of the housekeeping columns have only one entry for all the rows (subjects), __the tag-model used for each subject is not exactly the same__ - most of which are ST15, and a few are of a slightly different model. __Will this create some problem for the data collected?__ We will find out later!

Now let's clean up our reference table for further use. 

In [ ]:
whale_ref_clean = whale_ref[["animal-id","deploy-on-date","deploy-off-date",
                             "deploy-on-latitude","deploy-on-longitude","study-site",
                             "tag-manufacturer-name","tag-model"]]
whale_ref_clean.head(3)

Now let's move on to the juicer table! Repeat the same procedure done to the reference table.

In [ ]:
whale_data = pd.read_csv("../data/Blue_whales_Argos.csv")
whale_data.head(3)

In [ ]:
whale_data.shape

In [ ]:
whale_data.columns

Combined with the column information provided in the README.txt file, we can find a few columns that provide informations that will be useful to us 
- *timestamp*: The time point that a sensory measurement was taken
- *locaetion-long* & *location-lat*: the start location of monitoring
- *individual-local-identifier*: same as "animal-id" in reference table
- *manually-marked-outlier*: marked TRUE if "visible" marked FALSE

Some other technical measure associated with satellite signal receiving process that may be helpful for us to measure/visualize the credulity of data: 
- *argos:best-level*: Best signal strength
- *argos:calcul-freq*: Calculated frequency
- *argos:iq*: indicates transmitter oscillator frequency drift between two satellite passe
- *argos:nb-mes-120*: The number of messages received by the satellite at a signal strength greater than -120 decibels
- *sensor-type*: type of tracking sensor used (also appeared in reference table)



After identifying what we need and do not need, let us do some clean up and re-naming.

In [ ]:
whale_data_clean = whale_data[['individual-local-identifier','timestamp','location-lat', 
                               'location-long', 'manually-marked-outlier','argos:best-level', 
                               'argos:calcul-freq', 'argos:iq','argos:nb-mes',
                               'argos:nb-mes-120', 'sensor-type']]
whale_data_clean.columns = ['animal-id','timestamp','location-lat', 'location-long', 
                               'outlier','argos:best-level', 'argos:calcul-freq', 'argos:iq',
                               'argos:no-mes-rec','argos:no-mes-rec-120', 'sensor-type']
whale_data_clean.head()

After cleaning up the two tables, let's do some basic grouping and counting using __`pd.groupby`__. This allows us to understands the values in some of the parameters.

In [ ]:
whale_ref_clean.groupby('study-site').count()

In [ ]:
whale_data_clean.groupby('outlier').count()

__Outlier__: There are 471 out of 16249 records marked as outlier.

In [ ]:
whale_data_clean.groupby(['sensor-type']).count()

__sensor-type__: most of the data were recorded using argos-doppler-shift method

### 2nd-Degree Clean-up: Marking and Erasing the Outliers

Reading more closely into the reference table, I noticed that there are some individuals who were deployed on the tracker very shortly before they were taken off. It is best to leave these individuals out of the focus of our attention. <br>
Therefore, I decided to make a column that marks the how long the tracker was planted on each individuals. To do this, I need to make some changes of the data type first.

In [ ]:
whale_ref_clean.dtypes

Note that __`deploy-on-date`__ and __`deploy-off-date`__ are of type __`object`__. We need to change them into type __`datetime`__ using function __`pd.to_datetime()`__.

In [ ]:
whale_ref_clean['deploy-on-date'] = pd.to_datetime(whale_ref_clean['deploy-on-date'])
whale_ref_clean['deploy-off-date'] = pd.to_datetime(whale_ref_clean['deploy-off-date'])
whale_ref_clean.dtypes

In [ ]:
whale_ref_clean.insert(3,'monitor-duration', 
                       whale_ref_clean['deploy-off-date'] 
                        - whale_ref_clean['deploy-on-date'])
whale_ref_clean.head(3)

In [ ]:
whale_ref_clean.dtypes

Now that we have added column named __`monitor-duration`__, we can easily sort the reference table with respect to this characteristic.

In [ ]:
whale_ref_clean.sort_values('monitor-duration')

From the sorted table above, we can see that the duration in which trackers were kept on a  blue whale range from 2 hours to 503 days! We would definitely remove the poor whales who only had the tracker on for a couple hours from our data :( Let's create a histogram to visualize the __`monitor-duration`__ column.

In [ ]:
(whale_ref_clean['monitor-duration'] / pd.Timedelta(hours=1)).hist(bins = 20)
plt.xlabel('Time between deploy on and off tracker (hour)')
plt.ylabel('# of individuals');

It seems like that there are more than __50__ out of __143__ whales who wore the tracker for less than 12500 / 20 = 625 hours (about a month). Now let's look more closely on the portion below 480 hours (20 days)

In [ ]:
(whale_ref_clean['monitor-duration'] / pd.Timedelta(hours=1)).hist(bins = range(0, 480, 24))
plt.xlabel('Time between deploy on and off tracker (hour)')
plt.ylabel('# of individuals');

Judging from the zoomed-in histogram, it seems that there are 11 + 4 + 7 = 22 whales who wore the tracker for less than 72 hours (3 days) and need to be excluded from our data. 

Although we could go ahead and remove the outliers now, it would be more efficient if we cross reference with __`outliers`__ in the data table. To accomplish this, we need to merge the two tables into a big one using __`pd.merge`__.

In [ ]:
whale = pd.merge(whale_data_clean,
                 whale_ref_clean[['animal-id', 'deploy-on-date', 'deploy-off-date', 
                                  'monitor-duration', 'deploy-on-latitude', 
                                  'deploy-on-longitude', 'study-site']],
                 on='animal-id')
whale = whale[['animal-id', 'timestamp', 'location-lat', 'location-long', 'outlier',
               'deploy-on-date', 'deploy-off-date', 'monitor-duration', 'deploy-on-latitude', 
               'deploy-on-longitude', 'study-site', 'sensor-type', 'argos:best-level', 
               'argos:calcul-freq','argos:no-mes-rec', 'argos:no-mes-rec-120']]
whale.head()

Once we have a merge table, we can select records that are 1) not marked an outlier, and 2) not from an whale wearing tracker for less than 3 days

In [ ]:
whale_clean = whale.loc[(whale['outlier'] != True) 
                        & ((whale['monitor-duration']/pd.Timedelta(days=1)) > 3)]
whale_clean.head()

In [ ]:
whale_clean.shape

## Part 2: Visualization

### Different visualization attempts (and 3rd-Degree Clean-up, if needed)

Now that we have cleaned records, we can do some basic manipulations to visualize the records. 

In [ ]:
whale_clean.plot.scatter(x='location-long', y='location-lat')

To visualize the location records as points on a world map, let's draw a special scatter plot using __GeoPandas__. __GeoPandas__ extends the __Pandas__ package and allows for data analayis in the spatial and geometric setting. To install this package, use either __Anaconda Navigator__ or clone from its GitHub repo. Refer to the [official installation guide][1].

[1]: http://geopandas.org/install.html

btw the GeoPandas installation process is taking SO LONG

In [ ]:
import geopandas as gpd

xxxxxxxx SKIP geopanda section for now xxxxxxxx

It would also be cool to see how many movement data were recorded each year. First let's change __`timestamp`__ column into type __`pd.datetime`__ as well.

In [ ]:
whale_clean.dtypes

In [ ]:
whale_clean['timestamp'] = pd.to_datetime(whale_clean['timestamp'])
whale_clean.dtypes

Now let's plot a histogram with respect to year.

In [ ]:
whale_clean['timestamp'].dt.year.plot(kind = 'bar')
plt.xlabel('Year')
plt.ylabel('# of Records');

Note that there are no data recorded from 1997-1998

In [ ]:
pd.to_datetime(whale['timestamp']).dt.year.hist(bins = range(1993,2009,1))

In [ ]:
whale_clean['timestamp'].dt.month.hist(bins = range(1,13,1))